In [1]:
############################################ import libraries ############################################
import pandas as pd 
import numpy as np 
import warnings 
warnings.filterwarnings("ignore")
import os
from datetime import datetime, timedelta
from resampling_data import DataProcessor

In [2]:
#################################################### resampling equity data ####################################################
base_directory = r'C:\Users\user\Desktop\futuredata\spot_data'

# Read the CSV file into a DataFrame without header and set column names
directory1 = os.path.join(base_directory, 'ADANIENT.csv')

# Read the CSV file into a DataFrame
input_data = pd.read_csv(directory1, header=None, names=['DATE', 'TIME', 'TICK_PRICE', 'VOLUME', 'OPEN_INTEREST'])

# Create an instance of the DataProcessor class
processor = DataProcessor(input_data)

# Read CSV data with header
df_with_header = processor.read_csv_with_header()

# Preprocess the data
processor.preprocess_data()

# Resample the data to 5-second intervals(By default the interval is 5 seconds)
# If you want to change the interval you can pass the interval as a parameter to the resample_data method like 1T for 1 minute AND 1H for 1 hour
ohlc_data = processor.resample_data(interval='15T')

# Print the resulting OHLC data
print(ohlc_data)

                TIMESTAMP     open     high      low    close
0     2018-10-10 09:15:00   136.50   137.35   135.00   136.40
1     2018-10-10 09:30:00   136.35   136.70   135.25   135.95
2     2018-10-10 09:45:00   136.00   136.30   135.70   135.70
3     2018-10-10 10:00:00   135.80   137.95   135.80   137.50
4     2018-10-10 10:15:00   137.35   138.50   137.10   138.00
...                   ...      ...      ...      ...      ...
34726 2024-02-29 14:30:00  3255.05  3264.55  3247.25  3262.80
34727 2024-02-29 14:45:00  3262.90  3268.05  3256.00  3261.55
34728 2024-02-29 15:00:00  3262.45  3288.00  3260.25  3286.15
34729 2024-02-29 15:15:00  3286.20  3304.75  3285.65  3299.90
34730 2024-02-29 15:30:00  3285.40  3285.40  3285.40  3285.40

[34731 rows x 5 columns]


In [3]:
################################################################# stock split ########################################################
"""
# Convert 'TIMESTAMP' column to datetime if it's not already
ohlc_data['TIMESTAMP'] = pd.to_datetime(ohlc_data['TIMESTAMP'])

# Define the stock split dates
first_split_date = pd.Timestamp('2019-09-27')
second_split_date = pd.Timestamp('2021-06-22')

# Adjust prices before the first split date (divide by 4)
ohlc_data.loc[ohlc_data['TIMESTAMP'] < first_split_date, ['open', 'high', 'low', 'close']] /= 4

# Adjust prices between the first and second split date (divide by 2)
ohlc_data.loc[(ohlc_data['TIMESTAMP'] >= first_split_date) & (ohlc_data['TIMESTAMP'] < second_split_date), ['open', 'high', 'low', 'close']] /= 2

# Create the final DataFrame `df` with adjusted prices
df = pd.concat([
    ohlc_data[ohlc_data['TIMESTAMP'] < second_split_date][['TIMESTAMP', 'open', 'high', 'low', 'close']],
    ohlc_data[ohlc_data['TIMESTAMP'] >= second_split_date][['TIMESTAMP', 'open', 'high', 'low', 'close']]
])

# Reset index for `df` if needed
df.reset_index(drop=True, inplace=True)

# Print the results if desired
print("Final DataFrame `df` after adjustments:")
print(df)

"""

'\n# Convert \'TIMESTAMP\' column to datetime if it\'s not already\nohlc_data[\'TIMESTAMP\'] = pd.to_datetime(ohlc_data[\'TIMESTAMP\'])\n\n# Define the stock split dates\nfirst_split_date = pd.Timestamp(\'2019-09-27\')\nsecond_split_date = pd.Timestamp(\'2021-06-22\')\n\n# Adjust prices before the first split date (divide by 4)\nohlc_data.loc[ohlc_data[\'TIMESTAMP\'] < first_split_date, [\'open\', \'high\', \'low\', \'close\']] /= 4\n\n# Adjust prices between the first and second split date (divide by 2)\nohlc_data.loc[(ohlc_data[\'TIMESTAMP\'] >= first_split_date) & (ohlc_data[\'TIMESTAMP\'] < second_split_date), [\'open\', \'high\', \'low\', \'close\']] /= 2\n\n# Create the final DataFrame `df` with adjusted prices\ndf = pd.concat([\n    ohlc_data[ohlc_data[\'TIMESTAMP\'] < second_split_date][[\'TIMESTAMP\', \'open\', \'high\', \'low\', \'close\']],\n    ohlc_data[ohlc_data[\'TIMESTAMP\'] >= second_split_date][[\'TIMESTAMP\', \'open\', \'high\', \'low\', \'close\']]\n])\n\n# Reset in

In [4]:
#################################################### calculate EMA ####################################################
df = ohlc_data.copy()
i = 14
# Calculate EMA for high and low
df['ema_high'] = df['high'].ewm(span=i, adjust=False).mean()
df['ema_low'] = df['low'].ewm(span=i, adjust=False).mean()
# Initialize 'signal' column with 0
df['signal'] = 0

# Iterate through the DataFrame
for i in range(len(df)):
    if df['close'].iloc[i] > df['ema_high'].iloc[i]:
        df.at[i, 'signal'] = 1
    elif df['close'].iloc[i] < df['ema_low'].iloc[i]:
        df.at[i, 'signal'] = -1

# Apply forward fill to propagate the last valid observation forward
df['signal'] = df['signal'].replace(0, method='ffill')

# Shift the 'signal' column by 1 to move the signals to the next day
df['signal'] = df['signal'].shift(1)

In [5]:
########################################### calculate expiry date and position ###########################################
# Convert 'TIMESTAMP' to datetime
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])

# Initialize 'Expiry_Date' with None
df['Expiry_Date'] = None

# Iterate through the DataFrame
for i in range(len(df)):
    date = df['TIMESTAMP'].iloc[i].date()
    day = date.day
    month = date.month
    year = date.year
    if month == 12:
        if day < 20:
            df.at[i, 'Expiry_Date'] = f'{str(year)[2:4]}DECFUT'
        else:
            year += 1
            df.at[i, 'Expiry_Date'] = f'{str(year)[2:4]}JANFUT'
    else:
        if day < 20:
            df.at[i, 'Expiry_Date'] = f'{str(year)[2:4]}{date.strftime("%b").upper()}FUT'
        else:
            next_month_date = date + pd.DateOffset(months=1)
            df.at[i, 'Expiry_Date'] = f'{str(next_month_date.year)[2:4]}{next_month_date.strftime("%b").upper()}FUT'

# Initialize 'position' column with 0
df['position'] = 0

# Iterate through the DataFrame
for i in range(1, len(df)):
    if df['signal'].iloc[i] == 1 and df['signal'].iloc[i-1] == 0:
        df.at[i, 'position'] = 1
    elif df['signal'].iloc[i] == -1 and df['signal'].iloc[i-1] == 0:
        df.at[i, 'position'] = -1
    elif df['signal'].iloc[i] == -1 and df['signal'].iloc[i-1] == 1:
        df.at[i, 'position'] = -1
    elif df['signal'].iloc[i] == 1 and df['signal'].iloc[i-1] == -1:
        df.at[i, 'position'] = 1
              

In [6]:
###################################################################### logic to get future data ########################################################
directory = r'C:\Users\user\Desktop\futuredata'
df['price'] = None

for i in range(len(df)):
    # Ticker symbol
    ticker = 'ADANIENT'
    df.at[i, 'contract_name'] = ticker + df.at[i, 'Expiry_Date']
    df.at[i, 'date'] = df.at[i, 'TIMESTAMP'].date()
    df.at[i, 'time'] = df.at[i, 'TIMESTAMP'].time()
    
    if df.at[i, 'position'] == 1 or df.at[i, 'position'] == -1:
        base_directory = os.path.join(directory, df.at[i, 'contract_name'] + '.csv')
        #print(f"Processing file: {base_directory}")
        
        if os.path.exists(base_directory):
            data = pd.read_csv(base_directory)
            #print(f"File {base_directory} loaded successfully.")
            
            # Add the header to the future contract data
            data.columns = ['DATE', 'TIME', 'TICK_PRICE', 'VOLUME', 'OPEN_INTEREST']
            
            # Correctly parse the DATE column
            data['DATE'] = pd.to_datetime(data['DATE'], format='%Y%m%d').dt.date
            data['TIME'] = pd.to_datetime(data['TIME'], format='%H:%M:%S').dt.time
            
            # Debug: Print the first few rows of the data file
            #print("Data file preview:")
            #print(data.head())

            target_datetime = pd.to_datetime(df.at[i, 'date'].strftime('%Y-%m-%d') + ' ' + df.at[i, 'time'].strftime('%H:%M:%S'))
            
            # Combine 'DATE' and 'TIME' into a single 'DATETIME' column in data
            data['DATETIME'] = pd.to_datetime(data['DATE'].astype(str) + ' ' + data['TIME'].astype(str))
            
            # Find the row with the nearest time
            nearest_idx = (data['DATETIME'] - target_datetime).abs().argsort().iloc[0]
            nearest_row = data.iloc[nearest_idx]
            
            # Check if the nearest row date matches the target date
            if nearest_row['DATETIME'].date() == target_datetime.date():
                df.at[i, 'price'] = nearest_row['TICK_PRICE']
                #print(f"Price found: {df.at[i, 'price']}")
                #break  # Break the loop after finding the first price
print("Completed processing.")


Completed processing.


In [15]:
#df.to_csv('data.csv', index=False)

In [16]:
############################################# filterig data ########################################################
# Filter the DataFrame where 'price' is not None
filtered_df = df[df['price'].notna()][['position', 'date', 'time', 'contract_name', 'price']]

filtered_df.reset_index(drop=True, inplace=True)

# Assigning the Lot Size
filtered_df['lot_size'] = 300

# Display the new DataFrame
print(filtered_df)

      position        date      time     contract_name   price  lot_size
0            1  2018-10-10  10:15:00  ADANIENT18OCTFUT  137.55       300
1           -1  2018-10-11  10:00:00  ADANIENT18OCTFUT   138.3       300
2            1  2018-10-11  11:45:00  ADANIENT18OCTFUT  142.75       300
3           -1  2018-10-11  14:00:00  ADANIENT18OCTFUT   140.6       300
4            1  2018-10-12  09:30:00  ADANIENT18OCTFUT  143.15       300
...        ...         ...       ...               ...     ...       ...
1825         1  2024-02-23  12:30:00  ADANIENT24MARFUT  3312.8       300
1826        -1  2024-02-27  11:00:00  ADANIENT24MARFUT  3330.6       300
1827         1  2024-02-29  10:00:00  ADANIENT24MARFUT  3269.0       300
1828        -1  2024-02-29  10:15:00  ADANIENT24MARFUT  3250.0       300
1829         1  2024-02-29  11:15:00  ADANIENT24MARFUT  3278.2       300

[1830 rows x 6 columns]


In [17]:
# Initialize columns
filtered_df['entry_price'] = None
filtered_df['exit_price'] = None
filtered_df['pnl'] = None

# Calculate entry_price, exit_price, and pnl
for i in range(len(filtered_df) - 1):  # Loop until second last index
    if filtered_df.at[i, 'position'] == 1:
        filtered_df.at[i, 'entry_price'] = filtered_df.at[i, 'price']
        filtered_df.at[i, 'exit_price'] = filtered_df.at[i + 1, 'price']
        filtered_df.at[i, 'pnl'] = (filtered_df.at[i + 1, 'price'] - filtered_df.at[i, 'price']) * filtered_df.at[i, 'lot_size']
        
    elif filtered_df.at[i, 'position'] == -1:
        filtered_df.at[i, 'entry_price'] = filtered_df.at[i, 'price']
        filtered_df.at[i, 'exit_price'] = filtered_df.at[i + 1, 'price']
        filtered_df.at[i, 'pnl'] = (filtered_df.at[i, 'price'] - filtered_df.at[i + 1, 'price']) * filtered_df.at[i, 'lot_size']

# Drop the last row as it won't have an exit_price
filtered_df = filtered_df.drop(filtered_df.index[-1])

In [18]:
filtered_df

,position,date,time,contract_name,price,lot_size,entry_price,exit_price,pnl
0,1,2018-10-10,10:15:00,ADANIENT18OCTFUT,137.55,300,137.55,138.3,225.0
1,-1,2018-10-11,10:00:00,ADANIENT18OCTFUT,138.3,300,138.3,142.75,-1335.0
2,1,2018-10-11,11:45:00,ADANIENT18OCTFUT,142.75,300,142.75,140.6,-645.0
3,-1,2018-10-11,14:00:00,ADANIENT18OCTFUT,140.6,300,140.6,143.15,-765.0
4,1,2018-10-12,09:30:00,ADANIENT18OCTFUT,143.15,300,143.15,173.0,8955.0
...,...,...,...,...,...,...,...,...,...
1824,-1,2024-02-23,10:30:00,ADANIENT24MARFUT,3264.1,300,3264.1,3312.8,-14610.0
1825,1,2024-02-23,12:30:00,ADANIENT24MARFUT,3312.8,300,3312.8,3330.6,5340.0
1826,-1,2024-02-27,11:00:00,ADANIENT24MARFUT,3330.6,300,3330.6,3269.0,18480.0
1827,1,2024-02-29,10:00:00,ADANIENT24MARFUT,3269.0,300,3269.0,3250.0,-5700.0


In [19]:
#filtered_df.to_csv('filtered_df.csv', index=False)

In [20]:
# Convert date and time to strings and combine into a single datetime column
filtered_df['date'] = filtered_df['date'].astype(str)
filtered_df['time'] = filtered_df['time'].astype(str)
filtered_df['datetime'] = pd.to_datetime(filtered_df['date'] + ' ' + filtered_df['time'])

# Calculate the final capital
initial_capital = 800000
filtered_df['final_capital'] = initial_capital + filtered_df['pnl'].cumsum()

In [21]:
# Function to calculate overall profit 
def calculate_overall_profit(data, initial_capital):
    final_capital = data['final_capital'].iloc[-1]
    return final_capital - initial_capital

# Function to calculate overall profit percentage
def calculate_overall_profit_percentage(data, initial_capital):
    final_capital = data['final_capital'].iloc[-1]
    return ((final_capital - initial_capital) / initial_capital) * 100

# Function to calculate maximum drawdown
def calculate_maximum_drawdown(data):
    max_data = data['final_capital'].expanding(min_periods=1).max()
    drawdown = (data['final_capital'] - max_data) / max_data
    return drawdown.min()

# Function to calculate maximum drawdown percentage
def calculate_maximum_drawdown_percentage(data):
    max_data = data['final_capital'].expanding(min_periods=1).max()
    drawdown = (data['final_capital'] - max_data) / max_data
    return drawdown.min() * 100

# Function to calculate yearly profit
def calculate_yearly_profit(data):
    data['year'] = data['datetime'].dt.year
    yearly_capital = data.groupby('year')['final_capital'].agg(['first', 'last'])
    yearly_profit = yearly_capital['last'] - yearly_capital['first']
    return yearly_profit

# Function to calculate yearly profit percentage
def calculate_yearly_profit_percentage(data):
    yearly_profit = calculate_yearly_profit(data)
    data['year'] = data['datetime'].dt.year
    yearly_capital = data.groupby('year')['final_capital'].agg(['first', 'last'])
    yearly_profit_percentage = (yearly_profit / yearly_capital['first']) * 100
    return yearly_profit_percentage

# Function to calculate yearly drawdown
def calculate_yearly_drawdown(data):
    yearly_drawdown = {}
    for year, group in data.groupby('year'):
        max_data = group['final_capital'].expanding().max()
        drawdown = (group['final_capital'] - max_data) / max_data
        yearly_drawdown[year] = drawdown.min()
    return pd.Series(yearly_drawdown)

# Function to calculate yearly drawdown percentage
def calculate_yearly_drawdown_percentage(data):
    yearly_drawdown_percentage = {}
    for year, group in data.groupby('year'):
        max_data = group['final_capital'].expanding().max()
        drawdown = (group['final_capital'] - max_data) / max_data
        yearly_drawdown_percentage[year] = drawdown.min() * 100
    return pd.Series(yearly_drawdown_percentage)

# Function to calculate CAGR
def calculate_cagr(data, initial_capital):
    start_value = initial_capital
    end_value = data['final_capital'].iloc[-1]
    n_years = (data['datetime'].iloc[-1] - data['datetime'].iloc[0]).days / 365.25
    cagr = (end_value / start_value) ** (1 / n_years) - 1
    return cagr * 100

data = filtered_df
overall_profit = calculate_overall_profit(data, initial_capital)
overall_percentage = calculate_overall_profit_percentage(data, initial_capital)
mdd = calculate_maximum_drawdown(data)
mdd_percentage = calculate_maximum_drawdown_percentage(data)
yearly_profit = calculate_yearly_profit(data)
yearly_profit_percentage = calculate_yearly_profit_percentage(data)
yearly_drawdown = calculate_yearly_drawdown(data)
yearly_drawdown_percentage = calculate_yearly_drawdown_percentage(data)
cagr = calculate_cagr(data, initial_capital)



In [22]:
# Display the results
print("Overall Profit:", overall_profit)
print("Overall Profit Percentage:", overall_percentage)
print("Maximum Drawdown:", mdd)
print("Maximum Drawdown Percentage:", mdd_percentage)
print("Yearly Profit:", yearly_profit)
print("Yearly Profit Percentage:", yearly_profit_percentage)
print("Yearly Drawdown:", yearly_drawdown)
print("Yearly Drawdown Percentage:", yearly_drawdown_percentage)
print("CAGR:", cagr)

Overall Profit: 1450455.0000000014
Overall Profit Percentage: 181.3068750000002
Maximum Drawdown: -0.17985963113112283
Maximum Drawdown Percentage: -17.985963113112284
Yearly Profit: year
2018       9825.0
2019      12945.0
2020      16935.0
2021     164460.0
2022     181335.0
2023    1243605.0
2024    -173670.0
dtype: object
Yearly Profit Percentage: year
2018       1.22778
2019      1.599857
2020       2.06126
2021     19.597352
2022     18.064224
2023    105.098117
2024     -7.164235
dtype: object
Yearly Drawdown: 2018   -0.005299
2019   -0.017013
2020   -0.025262
2021   -0.108613
2022   -0.179860
2023   -0.072903
2024   -0.081075
dtype: float64
Yearly Drawdown Percentage: 2018    -0.529931
2019    -1.701299
2020    -2.526172
2021   -10.861334
2022   -17.985963
2023    -7.290329
2024    -8.107460
dtype: float64
CAGR: 21.161713678322116
